## deploy

In [17]:
import pandas as pd
import re
import os
import numpy as np

In [2]:
path_dir = r'C:\Users\ydjoe\Documents\nlp\dialog-bAbI-tasks'
src_dir = os.path.join(path_dir,'txt files')
dst_tmp_dir = os.path.join(path_dir,'csv files temp')
dst_dir = os.path.join(path_dir,'csv files')

lst_files = os.listdir(src_dir)

In [3]:
iter_tp = 0
for iter_fl in lst_files:
    print(iter_tp,iter_fl)
    iter_tp+=1

0 dialog-babi-candidates.txt
1 dialog-babi-kb-all.txt
2 dialog-babi-task1-API-calls-dev.txt
3 dialog-babi-task1-API-calls-trn.txt
4 dialog-babi-task1-API-calls-tst-OOV.txt
5 dialog-babi-task1-API-calls-tst.txt
6 dialog-babi-task2-API-refine-dev.txt
7 dialog-babi-task2-API-refine-trn.txt
8 dialog-babi-task2-API-refine-tst-OOV.txt
9 dialog-babi-task2-API-refine-tst.txt
10 dialog-babi-task3-options-dev.txt
11 dialog-babi-task3-options-trn.txt
12 dialog-babi-task3-options-tst-OOV.txt
13 dialog-babi-task3-options-tst.txt
14 dialog-babi-task4-phone-address-dev.txt
15 dialog-babi-task4-phone-address-trn.txt
16 dialog-babi-task4-phone-address-tst-OOV.txt
17 dialog-babi-task4-phone-address-tst.txt
18 dialog-babi-task5-full-dialogs-dev.txt
19 dialog-babi-task5-full-dialogs-trn.txt
20 dialog-babi-task5-full-dialogs-tst-OOV.txt
21 dialog-babi-task5-full-dialogs-tst.txt
22 dialog-babi-task6-dstc2-candidates.txt
23 dialog-babi-task6-dstc2-dev.txt
24 dialog-babi-task6-dstc2-kb.txt
25 dialog-babi-task

## task 3

In [4]:
fl_no = 10
path_final = os.path.join(src_dir,lst_files[fl_no])
print(path_final)

C:\Users\ydjoe\Documents\nlp\dialog-bAbI-tasks\txt files\dialog-babi-task3-options-dev.txt


In [80]:
df = pd.DataFrame([])
path_to_use = path_final
ct = 0
with open(path_to_use) as topo_file:
    cnt = 0
    cnt_sub = 0
    ar_3,ar_4 = [],[]
    ar_1,ar_2 = [],[]
    ar_6 = []

    for line in topo_file:
        #if ct>120: break
        ct+=1

        line = line.strip()    
        if not line.strip(): continue

        init_num = re.findall(r'\d+',line)        

        if init_num[0]=='1':

            ar_5 = cnt
            cnt+=1

            if len(ar_3)==0: continue
            ar_6 = list(filter(None, ar_6))
            ar_6 = np.unique(np.array(ar_6)).tolist()
            ar_6 = " ".join(ar_6)
            ar_6 = '<KB> ' + ar_6 + ' '            
            ar_6 = [ar_6]*(len(ar_3)-1)
            ar_6.insert(0,'')
            #print(np.unique(np.array(ar_6)))
            data = list(
                zip(                 
                    ar_3,
                    ar_4,
                    ar_6,
                    [ar_5]*len(ar_3)
                    ))

            df = df.append(data,ignore_index=True)
            cnt_sub = 0
            ar_3,ar_4,ar_6 = [],[],[]
            continue

        line_splits = line[2:].split('\t')
        #print(len(line_splits),line_splits,line[2:].split(' '))
        if len(line_splits)!=2:
            line_splits = line[2:].split(' ')
            #print(line_splits)
            ar_6.append(line_splits[0])
            continue        
        ar_3.append(line_splits[0])
        ar_4.append(line_splits[1])
        cnt_sub+=1

df = df.rename(columns={
     0: 'Call',
     1: 'Response',
     2: 'KB',
     3: 'Count'
     })
print(df.shape)
fl_dst = lst_files[fl_no][:-3] + 'csv'
path_dst_tmp = os.path.join(dst_tmp_dir,fl_dst)
df.to_csv(path_dst_tmp,index=False)

(9969, 4)


In [84]:
df.head()

,Call,Response,KB,Count
0,good morning,hello what can i help you with today,,1
1,can you book a table,i'm on it,<KB> resto_madrid_cheap_british_2stars resto_m...,1
2,<SILENCE>,any preference on a type of cuisine,<KB> resto_madrid_cheap_british_2stars resto_m...,1
3,with british cuisine,where should it be,<KB> resto_madrid_cheap_british_2stars resto_m...,1
4,madrid please,how many people would be in your party,<KB> resto_madrid_cheap_british_2stars resto_m...,1


In [85]:
prev_count = 0
txt = []
for i in range(len(df)):    
    if df.iloc[i][3]!=prev_count:
        prev_count = df.iloc[i][3]
        txt.append('')
        continue
    txt.append('<context>')  
df['TAG'] = txt

In [86]:
TAGS = [

    '<profile> ',
    ' <context>',
    ' <user> ',
    ' <system> ',
    '<query> ']

c = 0
mn_ar_1 = []
mn_ar_2 = []
mn_ar_3 = []
prev_count = 0
prev_string = ''
for i in range(len(df)):    
    #if c > 100: break
    c+=1    

    if df.iloc[i][3]!=prev_count:
        prev_string = ''
        prev_count = df.iloc[i][3]
    
    a2 = df.iloc[i][0]
    a3 = df.iloc[i][1]    
    if prev_string=='':
            call_string = prev_string + TAGS[4]+ a2    
    else:
        call_string = prev_string +' '+TAGS[4]+ a2  
    prev_string = prev_string + TAGS[2]+  a2 + ' ' + TAGS[3] + a3
    mn_ar_1.append(call_string)
    mn_ar_2.append(a3)

ndf = pd.DataFrame(            
            list(zip(
                mn_ar_1,mn_ar_2                  
                )),

            columns =[
                'Call',
                'Response'
                ]
                ) 
ndf['Call'] = df['TAG'] + ndf['Call']
ndf['Call'] = df['KB'] + ndf['Call']

print(ndf.shape)
fl_dst = lst_files[fl_no][:-3] + '.csv'
path_dst = os.path.join(dst_dir,fl_dst)
ndf.to_csv(path_dst,index=False)
    

(9969, 2)


In [41]:
ndf.shape

(9969, 2)

In [78]:
ndf.head(10)

,Call,Response
0,<query> good morning,hello what can i help you with today
1,resto_madrid_cheap_british_2stars resto_madrid...,i'm on it
2,resto_madrid_cheap_british_2stars resto_madrid...,any preference on a type of cuisine
3,resto_madrid_cheap_british_2stars resto_madrid...,where should it be
4,resto_madrid_cheap_british_2stars resto_madrid...,how many people would be in your party
5,resto_madrid_cheap_british_2stars resto_madrid...,which price range are looking for
6,resto_madrid_cheap_british_2stars resto_madrid...,ok let me look into some options for you
7,resto_madrid_cheap_british_2stars resto_madrid...,what do you think of this option: resto_madrid...
8,resto_madrid_cheap_british_2stars resto_madrid...,sure let me find an other option for you
9,resto_madrid_cheap_british_2stars resto_madrid...,what do you think of this option: resto_madrid...


(5009, 2)


## loop define

In [87]:
def convert_to_df(path_final):
    df = pd.DataFrame([])
    path_to_use = path_final
    ct = 0
    with open(path_to_use) as topo_file:
        cnt = 0
        cnt_sub = 0
        ar_3,ar_4 = [],[]
        ar_1,ar_2 = [],[]
        ar_6 = []

        for line in topo_file:
            #if ct>120: break
            ct+=1

            line = line.strip()    
            if not line.strip(): continue

            init_num = re.findall(r'\d+',line)        

            if init_num[0]=='1':

                ar_5 = cnt
                cnt+=1

                if len(ar_3)==0: continue
                ar_6 = list(filter(None, ar_6))
                ar_6 = np.unique(np.array(ar_6)).tolist()
                ar_6 = " ".join(ar_6)
                ar_6 = '<KB> ' + ar_6 + ' '            
                ar_6 = [ar_6]*(len(ar_3)-1)
                ar_6.insert(0,'')
                #print(np.unique(np.array(ar_6)))
                data = list(
                    zip(                 
                        ar_3,
                        ar_4,
                        ar_6,
                        [ar_5]*len(ar_3)
                        ))

                df = df.append(data,ignore_index=True)
                cnt_sub = 0
                ar_3,ar_4,ar_6 = [],[],[]
                continue

            line_splits = line[2:].split('\t')
            #print(len(line_splits),line_splits,line[2:].split(' '))
            if len(line_splits)!=2:
                line_splits = line[2:].split(' ')
                #print(line_splits)
                ar_6.append(line_splits[0])
                continue        
            ar_3.append(line_splits[0])
            ar_4.append(line_splits[1])
            cnt_sub+=1

    df = df.rename(columns={
        0: 'Call',
        1: 'Response',
        2: 'KB',
        3: 'Count'
        })
    print(df.shape)
    fl_dst = lst_files[fl_no][:-3] + 'csv'
    path_dst_tmp = os.path.join(dst_tmp_dir,fl_dst)
    df.to_csv(path_dst_tmp,index=False)
    return df

TAGS = [

    '<profile> ',
    ' <context>',
    ' <user> ',
    ' <system> ',
    '<query> ']

def add_context(df):
    prev_count = 0
    txt = []
    for i in range(len(df)):    
        if df.iloc[i][2]!=prev_count:
            prev_count = df.iloc[i][2]
            txt.append('')
            continue
        txt.append('<context>')  
    df['TAG'] = txt
    return df



def parse_from_df(df):
    c = 0
    mn_ar_1 = []
    mn_ar_2 = []
    mn_ar_3 = []
    prev_count = 0
    prev_string = ''
    for i in range(len(df)):    
        #if c > 100: break
        c+=1    

        if df.iloc[i][2]!=prev_count:
            prev_string = ''
            prev_count = df.iloc[i][2]
        
        a2 = df.iloc[i][0]
        a3 = df.iloc[i][1]            
        if prev_string=='':
            call_string = prev_string + TAGS[4]+ a2    
        else:
            call_string = prev_string +' '+TAGS[4]+ a2    
        prev_string = prev_string + TAGS[2]+ a2  + TAGS[3] + a3
        mn_ar_1.append(call_string)
        mn_ar_2.append(a3)

    ndf = pd.DataFrame(            
                list(zip(
                    mn_ar_1,mn_ar_2                  
                    )),

                columns =[
                    'Call',
                    'Response'
                    ]
                    ) 

    ndf['Call'] = df['TAG'] + ndf['Call']
    print(ndf.shape)
    fl_dst = lst_files[fl_no][:-3] + '.csv'
    path_dst = os.path.join(dst_dir,fl_dst)
    ndf.to_csv(path_dst,index=False)   

In [90]:
for i in range(9,15):
    fl_no = i
    path_final = os.path.join(src_dir,lst_files[fl_no])
    print(i,path_final)

9 C:\Users\ydjoe\Documents\nlp\dialog-bAbI-tasks\txt files\dialog-babi-task2-API-refine-tst.txt
10 C:\Users\ydjoe\Documents\nlp\dialog-bAbI-tasks\txt files\dialog-babi-task3-options-dev.txt
11 C:\Users\ydjoe\Documents\nlp\dialog-bAbI-tasks\txt files\dialog-babi-task3-options-trn.txt
12 C:\Users\ydjoe\Documents\nlp\dialog-bAbI-tasks\txt files\dialog-babi-task3-options-tst-OOV.txt
13 C:\Users\ydjoe\Documents\nlp\dialog-bAbI-tasks\txt files\dialog-babi-task3-options-tst.txt
14 C:\Users\ydjoe\Documents\nlp\dialog-bAbI-tasks\txt files\dialog-babi-task4-phone-address-dev.txt


## loop run

In [91]:
for i in range(11,14):
    fl_no = i
    path_final = os.path.join(src_dir,lst_files[fl_no])
    df = convert_to_df(path_final)
    df = add_context(df)
    parse_from_df(df)
    

(9846, 4)
(9846, 2)
(9779, 4)
(9779, 2)
(10006, 4)
(10006, 2)


## task 3,5

In [62]:
fl_no = 18
path_final = os.path.join(src_dir,lst_files[fl_no])
print(path_final)

C:\Users\ydjoe\Documents\nlp\personalized-dialog-dataset\full\txt files\personalized-dialog-task5-full-dialogs-tst-OOV.txt


In [ ]:
df = pd.DataFrame([])
path_to_use = path_final
ct = 0
with open(path_to_use) as topo_file:
    
    cnt = 0
    cnt_sub = 0
    ar_1,ar_2,ar_3,ar_4 = [],[],[],[]
    ar_5,ar_6 = [],[]

    for line in topo_file:
        # if ct>120: break
        # ct+=1

        #print(line)

        line = line.strip()
    
        if not line.strip(): continue

        init_num = re.findall(r'\d+',line)
        #print(init_num)

        if init_num[0]=='1':

            line_splits = line[1:].split()   
            #print(line_splits)         
            ar_1.append(line_splits[0])
            ar_2.append(line_splits[1])
            ar_3.append(line_splits[2])
            ar_4.append(line_splits[3])
            ar_7 = cnt          
            cnt+=1
            
            if len(ar_5)==0: continue
            
            data = list(
                zip(
                    [ar_1[0]]*len(ar_5),
                    [ar_2[0]]*len(ar_5),
                    [ar_3[0]]*len(ar_5),
                    [ar_4[0]]*len(ar_5),
                    ar_5,
                    ar_6,
                    [ar_7]*len(ar_5)

                    ))

            df = df.append(data,ignore_index=True)
            cnt_sub = 0
            ar_5,ar_6 = [],[]
            ar_1.pop(0)
            ar_2.pop(0)
            ar_3.pop(0)
            ar_4.pop(0)
            continue

        
        line_splits = line[2:].split('\t')
        if len(line_splits)!= 2: continue
        #print(line_splits)
        ar_5.append(line_splits[0])
        ar_6.append(line_splits[1])
        cnt_sub+=1

df = df.rename(columns={
     0: 'Gender',
     1: 'Age',
     2: 'Food',
     3: 'Dish',
     4: 'Call',
     5: 'Response',
     6: 'Count'
     })
print(df.shape)

fl_dst = lst_files[fl_no][:-3] + 'csv'
path_dst_tmp = os.path.join(dst_tmp_dir,fl_dst)
df.to_csv(path_dst_tmp,index=False)

In [16]:
TAGS = [
    '<profile> ',
    ' <context>',
    ' <user> ',
    ' <system> ',
    ' <query> ']
    
c = 0
mn_ar_1 = []
mn_ar_2 = []
mn_ar_3 = []
prev_count = 0
prev_string = ''
for i in range(len(df)):    
    #if c > 100: break
    c+=1    

    if df.iloc[i][6]!=prev_count:
        prev_string = ''
        prev_count = df.iloc[i][6]
    
    a2 = df.iloc[i][4]
    a3 = df.iloc[i][5]    
    call_string = prev_string + TAGS[4]+ a2    
    prev_string = prev_string + TAGS[2]+ a2 + TAGS[3] + a3
    # print(a2)
    # print('call: ',call_string)
    # print('prev: ',call_string)
    # print(a3)
    to_append = TAGS[0]+ line
    mn_ar_1.append(call_string)
    mn_ar_2.append(a3)

ndf = pd.DataFrame(            
            list(zip(
                mn_ar_1,mn_ar_2                  
                )),

            columns =[
                'Call',
                'Response'
                ]
                ) 

df['Profile'] = df['Gender'] + ' ' +df['Age'] + ' ' + df['Food'] + ' ' + df['Dish']
ndf['Call'] = TAGS[0]+ df['Profile'] + TAGS[1] + ndf['Call']

fl_dst = lst_files[fl_no][:-3] + 'csv'
path_dst = os.path.join(dst_dir,fl_dst)
ndf.to_csv(path_dst,index=False)
    